<img src="images/notebook11_header.png" width="1024" alt="Python for Geospatial Data Science" style="border-radius:10px"/>

**Dr Gunnar Mallon** (g.mallon@rug.nl), *Department of Cultural Geography (Faculty of Spatial Science)*, *University of Groningen*

---

# Geospatial Data Analysis with GeoPandas

We have come to the end of the short introduction to Python for Geospatial Data Science. Well done for making it this far!! Hopefully, you are all excited about your newfound programming skills.

In this final section (we left the best for last), we are going to pull together all of what we have covered so far, and apply our knowledge to produce a map visualizing population changes by U.S. state between 2021 and 2022.

This file is largely a walk through on how to create cool maps with Python. At the end you will be set a big exercise that will test all your newly learned Python knowledge!

<img src="images/map.png" width="480" alt="Python for Geospatial Data Science" style="border-radius:10px"/>

## Setting Up Our Environment

First, ensure that you have downloaded the necessary datasets containing the population changes and the shapefiles delineating state boundaries. You will find these on Brightspace.

Make sure your data is organized in a logical folder structure to facilitate easy access. The project structure looks like this:
- Jupyter notebook file (this file).
- A folder named `population_estimates` with the CSV file of population data.
- A folder named `shapefiles` with the respective geographic shapefiles.

Now, let's install the libraries necessary for our geospatial analysis if you haven't done so.

```
!pip install geopandas matplotlib
```

Next, we\'ll import the libraries required for our analysis:

```python
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
```

You will notice two new packages. Matplotlib is a library for plotting diagrams in Python and I'm sure you can guess what Geopandas is.

## Loading and Preparing the Data

Let us start by setting up the paths to our CSV data and shapefile.

```python
data_path = 'population_estimates/NST-EST2022-POPCHG2020_2022.csv'
shape_path = 'shapefiles/cv_2018_us_state_500k.shp'
```

Load the population data into a Pandas DataFrame and examine its contents:


```python
# Read the csv file into the DataFrame
df = pd.read_csv(data_path)
```

To have a look at the first few rows of a DataFrame you can use the `DataFrame.head()` method. This is an essential method to have a quick look at a DataFrame. I suggest that after each change to the DataFrame, you have a quick look at it using `head()`.

```python

# Let's check the first few rows of the DataFrame
df.head()
```

After inspecting the DataFrame, we find that we can drop the column we do not need and rename the others for clarity:

```python
# DataFrame.drop let's us remove a column from the DataFrame. In this case we're dropping ESTIMATESBASE2020
df.drop(columns=['ESTIMATESBASE2020'], inplace=True)

# You can rename the columns in your DataFrame but simply setting the DataFrame.columns to new values
df.columns = ['State', 'Population_2021', 'Population_2022']
```

Make sure to have a look at the new DataFrame

```python
df.head()
```

The populations are read as strings and if we want to work with them we'll need to convert these to integers by using the `astype(int)` method:

```python
df['Population_2021'] = df['Population_2021'].astype(int)
df['Population_2022'] = df['Population_2022'].astype(int)
```

Next, we'll calculate the population change percentage from 2021 to 2022 and add it as a new column. This will be the data that we are going to plot on the map.

```python
df['Population_Change'] = ((df['Population_2022'] - df['Population_2021']) / df['Population_2022']) * 100
```

Remember to look at your new DataFrame using `df.head()`

## Integrating the Geographic Data

With GeoPandas, we can read in our shapefile directly and explore the data:

```python
shape_data = gpd.read_file(shape_path)
shape_data.head()
```

Once we have both the population DataFrame and the geospatial DataFrame, we can merge them. Remember, you looked at this in the Pandas notebook. 

When you look at the `head()` of both DataFrames, you'll see that the U.S. States are listed under different column names. We could rename the columns, or use the `left_on` and `right_on` parameters.

```python
shape_data = shape_data.merge(df, left_on='NAME', right_on='State', how='left')
```

Here we merge the data based on the **NAME** column in the shape_data DataFrame and the **State** column in the df DataFrame. Have a look what the `how='left'` means.

We will then drop any entries with missing geospatial data using the `dropna` method:

```python
shape_data.dropna(subset=['Population_Change'], inplace=True)
```

## Creating the Map

Ok, it's time for the fun 🎉 part. Let's create a cool map of the data! We will focus exclusively on the continental United States. To do that we need to exclude some rows of the island and remote states (Alaska, Hawaii, and Puerto Rico).

```python
# Exclude territories outside of the continental US, if necessary
exclude_territories = ['Alaska', 'Hawaii', 'Puerto Rico'] # Add any other territories if needed
shape_data = shape_data[~shape_data['State'].isin(exclude_territories)]
```

Let's break this down a little. After creating the list of territories to exclude, we have to remove them.

First we create a boolean mask of all rows where the **State** is included in the exclude_territories list. This will return `True` for the three territories, Alaska, Hawaii, and Puerto Rico. 

However, we want to exclude these. So we use the tilde ('~') to signify a **NOT** condition. Essentially this means that we get a `True` value for our boolean mask, when the row's **State** value is **NOT** in the list of exclusion territories.

Finally, we use the boolean mask to get the new shape_data.

The next part is from the MatplotLib library for creating plots. Don't worry if it looks a little intimidating. We'll break it into small bits.

We start by creating a new plot and we call it ax and then show it.

```python
# Create the plot and store it in 'ax'
ax = shape_data.plot(column='Population_Change')

# Use Matplotlib to show the plot
plt.show()
```

Pretty cool, wouldn't you say? I think so!

You can see that there are differences between the states representing changes in population, but we don't know what these mean. We need a legend! We can achieve this by adding the `legend=True` parameter to the method that creates the plot

```python
# Create the plot and store it in 'ax'
ax = shape_data.plot(
    column='Population_Change',
    legend=True
)

# Use Matplotlib to show the plot
plt.show()
```

So far so good. Now we know what the individual colors mean. But we can still do one better. We can turn off the axes because we are not really interested in longitude and latitude but in population change.

We can achieve this by calling `ax.set_axis_off()` before showing the plot.

```python
# Create the plot and store it in 'ax'
ax = shape_data.plot(
    column='Population_Change',
    legend=True
)

# Use Matplotlib to show the plot
ax.set_axis_off()
plt.show()
```

Much better! But we can do better still. Let's do the following changes:

- **Change the figure size**: We can use `figsize` for this. Let's make it 15:9
- **Reformat the legend**: The legend is a little distracting, let's put it below the map using the `legend_kwds` parameter
- **Change the colour scheme**: The colours look pretty cool, but I prefer them to go from Red to Blue rather than Yellow to Blue. We do this with the `cmap` parameter

If we implement all those changes we get:

```python
# Create the plot and store it in 'ax'
ax = shape_data.plot(
    column='Population_Change',
    legend=True,
    legend_kwds={
        'label': "Population Change (%)",
        'orientation': "horizontal",
        'format': "%0.2f"
    },
    figsize=(15, 9),
    cmap='RdBu',
)

# Use Matplotlib to show the plot
ax.set_axis_off()
plt.show()
```

One last thing...

The states seem to float a little, so let's put lines around them. We can achieve this with the `edgecolor` and `linewidth` methods. Edgecolor sets the colour of the edges of each polygon and linewidth sets the width of the border.

```python
# Create the plot and store it in 'ax'
ax = shape_data.plot(
    column='Population_Change',
    legend=True,
    legend_kwds={
        'label': "Population Change (%)",
        'orientation': "horizontal",
        'format': "%0.2f"
    },
    figsize=(15, 9),
    cmap='RdBu',
    edgecolor='black',
    linewidth=0.8
)

# Use Matplotlib to show the plot
ax.set_axis_off()
plt.show()
```

We can't forget adding a title to the graph

```python
ax.set_title('Population Change in the USA (2020 - 2021)', fontsize=18, weight='bold')
```

And there you have it. From 2021 to 2022, a lot of people moved from New York to Florida and Idaho (?). 

At this point, I'd like you to stand up and rejoice at what you have accomplished. You have loaded a data file and a shape file directly into a Python script. There you've done some data cleaning and merging (in Python!) and then plotted a pretty cool graph (in Python).

Excellent job!

## Final exercise

Now that you are a Python and Geopandas wizard, it's time to put your skills to the test. In the folder "exercise_data" you will find a shape file containing the world's countries and a csv file of containing data on the world's volcanic eruptions during the Holcone. Your task is to:

- **Produce a map showing the number of volcanic eruptions per country during the Holocene using Geopandas.**

Good luck and as always, if you get stuck or need help, please don't be shy to ask.

**Note**: If you are having difficulties preparing the csv file have a look at `DataFrame.groupby` and `DataFrame.reset_index`. If you are still having issues, I have created a summary csv file for you called 'eruptions_count_emergency.csv' for you to continue making the map - try getting to that stage on your own first.